In [1]:
import pickle
import random
import string
import time
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [ ]:
import os
os.chdir('../movies')
from movieLens import MovieLens

ml = MovieLens()

# Nuevos usuarios

Vamos a asociar un nombre de usuario y contraseña a cada uno de los userIds existentes en el dataset de ratings

In [ ]:
def create_user_password_dict(num_users):
    users = {}
    for i in range(num_users):
        user = f"user{i + 1}"
        password = ''.join(random.choices(string.ascii_letters + string.digits, k=12))
        users[i + 1] = {"user": user, "password": password}
    return users

num_users = 610
user_password_dict = create_user_password_dict(num_users)

# Show users' information
for user_id, data in user_password_dict.items():
    print(f"ID: {user_id} | User: {data['user']} | Password: {data['password']}")


In [ ]:
# Change the password of users 1 and 4
user_password_dict[1]["password"] = "123"
user_password_dict[4]["password"] = "456"

# # Show users' information
# for user_id, data in user_password.items():
#     print(f"ID: {user_id} | User: {data['user']} | Password: {data['password']}")

In [ ]:
# Save the dictionary
with open("../dashboard/user_password_dict.pkl", 'wb') as file:
        pickle.dump(user_password_dict, file)

Añadir nuevos usuarios al diccionario

In [ ]:
def add_new_user(username, password, users):
    for data in users.values():
        if data["user"] == username:
            print(f"El nombre de usuario '{username}' ya está en uso.")
            return

    # Obtener el último ID existente y aumentarlo en 1
    last_id = max(users.keys())
    new_id = last_id + 1

    # Agregar el nuevo usuario al diccionario
    users[new_id] = {"user": username, "password": password}
    print(f"Nuevo usuario agregado: ID: {new_id} | User: {username} | Password: {password}")

# Ejemplo de uso
username = input("Introduce el nombre de usuario: ")
password = input("Introduce la contraseña: ")

add_new_user(username, password, user_password_dict)

In [ ]:
existing_usernames = [user['user'] for user in user_password_dict.values()]
existing_usernames

def existing_username(username,user_password_dict):
    return username in [user['user'] for user in user_password_dict.values()]

existing_username("user5555",user_password_dict)

In [ ]:
# Load user-password database
with open('../dashboard/user_password_dict.pkl', 'rb') as file:
    user_password_dict = pickle.load(file)

In [ ]:
user_password_dict

In [ ]:
def get_user_id(username, user_password_dict):
    for user_id, user_data in user_password_dict.items():
        if user_data['user'] == username:
            return user_id
    return None  # Return None if the username is not found

user_id = get_user_id('user1', user_password_dict)
user_id

# QUIZ. Obtener las películas con mayor número de valoraciones

De las 50 películas más valoradas en total, se mostrarán aleatoriamente 15 de ellas a un nuevo usuario tras registrarse con el fin de conocer de forma superficial sus gustos cinematográficos y poder realizar recomendaciones personalizadas

In [ ]:
ratings = ml.ratings.copy()
ratings.head()

In [ ]:
k = 50
top_movies = ratings.groupby('movieId').size().nlargest(k).index.tolist()
top_movies_titles = [ml.movieID_to_name.get(movie_id, 'Unknown') for movie_id in top_movies]
top_movies_titles

Vamos a adaptar el nombre de las películas para que sea más sencillo de leer

In [ ]:
top_movies_titles = ['Forrest Gump',
 'The Shawshank Redemption',
 'Pulp Fiction',
 'The Silence of the Lambs',
 'The Matrix',
 'Star Wars: Episode IV - A New Hope',
 'Jurassic Park',
 'Braveheart',
 'Terminator 2: Judgment Day',
 "Schindler's List",
 'Fight Club',
 'Toy Story',
 'Star Wars: Episode V',
 'The Usual Suspects',
 'American Beauty',
 'Seven',
 'Independence Day',
 'Apollo 13',
 'Raiders of the Lost Ark',
 'The Fellowship of the Ring',
 'Star Wars: Episode VI',
 'The Godfather',
 'The Fugitive',
 'Batman',
 'Saving Private Ryan',
 ' The Two Towers',
 'The Return of the King',
 'Aladdin',
 'Fargo',
 'The Sixth Sense',
 'True Lies',
 'Twelve Monkeys',
 'The Lion King',
 'Speed',
 'Back to the Future',
 'Gladiator',
 'Shrek',
 'Men in Black',
 'Dances with Wolves',
 'Mission: Impossible',
 'Ace Ventura: Pet Detective',
 'Memento',
 'The Mask',
 'The Curse of the Black Pearl',
 'The Dark Knight',
 'Beauty and the Beast',
 'Alien',
 'Die Hard',
 'Die Hard: With a Vengeance',
 'Mrs. Doubtfire']

In [ ]:
top_movies_dict = {}
for movie_id, movie_title in zip(top_movies, top_movies_titles):
    top_movies_dict[movie_id] = movie_title
    
top_movies_dict

In [ ]:
# Save the dictionary
with open("../dashboard/top_movies_dict.pkl", 'wb') as file:
        pickle.dump(top_movies_dict, file)

Seleccionar de manera aleatoria 15 películas entre las 50 posibles

In [ ]:
import random

def randomly_select_movies(dictionary, quantity):
    selected_keys = random.sample(list(dictionary.keys()), quantity)
    selected_dictionary = {key: dictionary[key] for key in selected_keys}
    return selected_dictionary

selected_movies = randomly_select_movies(top_movies_dict, 15)
selected_movies

# Recomendaciones

In [ ]:
name_to_movieID = ml.name_to_movieID.copy()

with open("../dashboard/name_to_movieID.pkl", 'wb') as file:
        pickle.dump(name_to_movieID, file)

Vamos a ver cuales son las películas mejor valoradas. Para ello primero seleccionamos las 100 películas con mayor número de valoraciones, y entre ellas se seleccionan aquellas con mejor valoración media

In [ ]:
most_rated_movies = ratings.groupby('movieId').count().sort_values(by='rating', ascending=False).nlargest(100, 'rating')
most_rated_movies = most_rated_movies.reset_index()
ids_most_rated_movies = most_rated_movies['movieId']
ids_most_rated_movies

# Filter the ratings dataframe to obtain the information about the most rated mvoies
filtered_df = ratings[ratings['movieId'].isin(ids_most_rated_movies)]

# Compute the average rating of the movies
sorted_ratings = filtered_df.groupby('movieId').mean('rating').sort_values(by='rating', ascending=False).nlargest(10, 'rating')
sorted_ratings = sorted_ratings.reset_index()
sorted_movies = sorted_ratings['movieId']


sorted_movies_titles = [ml.movieID_to_name.get(movie_id, 'Unknown') for movie_id in sorted_movies]
sorted_movies_titles

sorted_movies_dict = {}
for movie_id, movie_title in zip(sorted_movies, sorted_movies_titles):
    sorted_movies_dict[movie_id] = movie_title
    
sorted_movies_dict

In [ ]:
# Common elements in top_movies and sorted_movies
list(set(top_movies_titles) & set(sorted_movies_titles))

Las recomendaciones para un nuevo usuario se  obtienen calculando la distancia entre usuarios y seleccionando aquellas recomendaciones que se hagan al usuario con gustos más semejantes al nuevo usuario.

## Cálculo de la distancia entre usuarios

En primer lugar vamos a añadir la información del nuevo usuario al conjunto de datos existente

In [ ]:
ratings = pd.read_csv('../dashboard/ratings.csv')
ratings.tail()

Añadir el nuevo usuario a la base de datos

In [ ]:
# Load new users' info
with open('../dashboard/valores_slider.pkl', 'rb') as file:
    valores_slider = pickle.load(file)
    
valores_slider

In [ ]:
userId = ratings['userId'].max()+1
timestamp = int(time.time())

# Add the slider information to the ratings dataframe
new_ratings = pd.DataFrame.from_dict(valores_slider, orient='index', columns=['rating'])
new_ratings.index.name = 'movieId'
new_ratings = new_ratings.reset_index()
new_ratings['userId'] = userId
new_ratings['timestamp'] = timestamp

# Reorder the columns in `df_new_ratings` to match the order of the existing DataFrame
new_ratings = new_ratings.reindex(columns=ratings.columns)
new_ratings

In [ ]:
# Concatenate the `ratings` DataFrame and `df_new_ratings` to add the new lines at the end
ratings = pd.concat([ratings, new_ratings], ignore_index=True)
ratings.to_csv('ratings.csv', index=False)
ratings

In [ ]:
def add_new_user():
    # Load new users' info
    with open('../dashboard/valores_slider.pkl', 'rb') as file:
        valores_slider = pickle.load(file)
        
    #ratings = pd.read_csv('ratings.csv')
    
    userId = ratings['userId'].max()+1
    timestamp = int(time.time())
    
    

    # Add the slider information to the ratings dataframe
    new_ratings = pd.DataFrame.from_dict(valores_slider, orient='index', columns=['rating'])
    new_ratings.index.name = 'movieId'
    new_ratings = new_ratings.reset_index()
    new_ratings['userId'] = userId
    new_ratings['timestamp'] = timestamp

    # Reorder the columns in new_ratings to match the order of the ratings DataFrame
    new_ratings = new_ratings.reindex(columns=ratings.columns)
    
    # Concatenate the ratings DataFrame and new_ratings to add the new lines at the end
    ratings = pd.concat([ratings, new_ratings], ignore_index=True)
    
    return ratings
    
    # Save the updated file
    # ratings.to_csv('ratings.csv', index=False)

In [ ]:
def find_most_similar_user(userId, distance_matrix):

    user_index = userId-1 # Restamos 1 porque los índices en Python comienzan desde 0

    # Calcular la distancia entre el usuario 611 y los demás usuarios
    user_distances = distance_matrix[user_index]

    # Encontrar el usuario más cercano (excluyendo al usuario mismo)
    most_similar_user_index = np.argmin(user_distances[np.nonzero(user_distances)])
    most_similar_user = most_similar_user_index + 1  # Sumamos 1 para obtener el índice real

    # Imprimir el índice del usuario más parecido
    print("El usuario más parecido al usuario "+str(userId)+" es:", most_similar_user) 
    
    with open('userId-to-recommend.pkl', 'wb') as archivo:
        pickle.dump(most_similar_user, archivo)

In [ ]:
def compute_distance_matrix(ratings):
    # Create a pivot table
    pivot_table = pd.pivot_table(ratings, values='rating', index='userId', columns='movieId')

    # Replace NaN values with 0
    pivot_table = pivot_table.fillna(0)
    
    # Compute the distance between users
    distances = pdist(pivot_table.values, metric='euclidean')

    # Create the distance matrix
    distance_matrix = squareform(distances)
    most_similar_user_index = find_most_similar_user(ratings['userId'].max(),distance_matrix)

In [ ]:
compute_distance_matrix(ratings)

In [2]:
with open('recommendations.pkl', 'rb') as file:
    recommendations = pickle.load(file)
recommendations[1]

[(778, 5.0),
 (356, 5.0),
 (4011, 5.0),
 (260, 5.0),
 (4973, 5.0),
 (1237, 5.0),
 (1204, 5.0),
 (177593, 5.0),
 (741, 4.981982623831946),
 (475, 4.979932392050417)]

# Recomendaciones usuarios existentes

In [ ]:
with open('../dashboard/recommendations.pkl', 'rb') as file:
        recommendations = pickle.load(file)
recommendations

In [ ]:
def get_movie_images(userId):

    numbers = [item[0] for item in recommendations[userId]]
    images = []
    for number in numbers:
        image_path = f"{number}.jpg"
        image = html.Img(src=dash.get_asset_url(image_path), style={"width": "110px", "height": "140px", "margin": "10px"})
        images.append(image)
    return images


In [ ]:
def get_user_preferences(userId, k, like=True):

    # Filter ratings for the specified userId and sort movies by rating in descending order
    user_ratings = ratings[ratings['userId'] == userId].copy()
    sorted_movies = user_ratings.sort_values(by='rating', ascending=False)

    if like:
        # Get the top k movies with highest ratings
        movieIds = list(sorted_movies.head(k).copy()['movieId'])
    
    # Get the top k movies with lowest ratings
    else:
        movieIds = list(sorted_movies.tail(k).copy()['movieId'])

    return movieIds

In [ ]:
numbers = get_user_preferences(1, 10, False)
numbers

In [ ]:
with open('../dashboard/recommendations.pkl', 'rb') as file:
    recommendations = pickle.load(file)

#recommendations